In [1]:
import pandas as pd
import pickle

In [2]:
with open(f'../models/final_models/model1.pickle', 'rb') as f:
    model1 = pickle.load(f)

In [4]:
with open(f'../models/final_models/model2.pickle', 'rb') as f:
    model2 = pickle.load(f)

In [5]:
with open(f'../models/final_models/model3.pickle', 'rb') as f:
    model3 = pickle.load(f)

In [7]:
import ZhongShan

with open(f'../models/AFL_pipeline_N.pickle', 'rb') as f:
    sanmin = pickle.load(f)
    
model3_COLS = sanmin.final_features['3']
model2_COLS = sanmin.final_features['2']
model1_COLS = sanmin.final_features['1']

model3_COLS = [x for x in model3_COLS if x not in ['3', '2', '1']]
model2_COLS = [x for x in model2_COLS if x not in ['3', '2', '1']]
model1_COLS = [x for x in model1_COLS if x not in ['3', '2', '1']]

In [8]:
import os
from collections import defaultdict as dd
manip_type = 'NormalisedData'

csv_list = os.listdir(f'../data/curated/{manip_type}')
csv_list.sort()

tally = dd(int)

data = pd.DataFrame()
for file in csv_list[1:]:
    if '2022' in file:
        data = pd.read_csv(f'../data/curated/{manip_type}/{file}')

        player = data['Player']
        pred3 = model3.predict(data[model3_COLS])
        pred2 = model2.predict(data[model2_COLS])
        pred1 = model1.predict(data[model1_COLS])
        pred = pd.DataFrame({'player': player, '3': pred3, '2': pred2, '1': pred1})

        three_votes = list(pred.sort_values('3', ascending = False)['player'])[0]

        two_votes = list(pred.sort_values('2', ascending = False)['player'])[0]
        if two_votes == three_votes:
            two_votes = list(pred.sort_values('2', ascending = False)['player'])[1]

        one_vote = list(pred.sort_values('1', ascending = False)['player'])[0]
        if one_vote in (three_votes, two_votes):
            one_vote = list(pred.sort_values('2', ascending = False)['player'])[1]
            
            if one_vote == two_votes:
                one_vote = list(pred.sort_values('2', ascending = False)['player'])[2]

        tally[three_votes] += 3
        tally[two_votes] += 2
        tally[one_vote] += 1
        

In [9]:
tally_list = list(tally.items())
tally_list.sort(key = lambda x:x[1], reverse=True)
tally_list

[('Clayton Oliver', 35),
 ('Patrick Cripps', 28),
 ('Lachie Neale', 22),
 ('Jeremy Cameron', 21),
 ('Touk Miller', 21),
 ('Christian Petracca', 20),
 ('Charlie Curnow', 20),
 ('Rory Laird', 20),
 ('Darcy Parish', 19),
 ('Sam Walsh', 17),
 ('Chad Warner', 17),
 ('Andrew Brayshaw', 16),
 ('Stephen Coniglio', 16),
 ('Taylor Walker', 16),
 ('Zachary Merrett', 16),
 ('Bailey Smith', 14),
 ('Thomas Liberatore', 14),
 ('Brad Crouch', 13),
 ('Hugh McCluggage', 13),
 ('Joshua Kelly', 12),
 ('Tom Hawkins', 12),
 ('Shai Bolton', 12),
 ('Tom Lynch', 12),
 ('Dion Prestia', 11),
 ('Max King', 11),
 ('Nick Larkey', 11),
 ('Callum Mills', 11),
 ('Josh Dunkley', 11),
 ('Luke Parker', 10),
 ('Tim Taranto', 10),
 ('Daniel Rich', 10),
 ('Karl Amon', 10),
 ('Will Brodie', 10),
 ('Todd Marshall', 9),
 ('Aaron Naughton', 9),
 ('Mitchell Lewis', 9),
 ('Peter Wright', 9),
 ('Jack Higgins', 9),
 ('Marcus Bontempelli', 9),
 ('Connor Rozee', 9),
 ('Oliver Wines', 9),
 ('Ben Keays', 8),
 ('George Hewett', 8),
 ('P